# Module 1: Reading in and processing Word documents (Focus Group data)

## Sourcing packages
- The textract package is used to read in the .docx files.
- The gensim package is used to fit prelimnary LDA models on the data and filter out words which are common to the majority of the identified topics.
- The nltk package is used to get an initial list of stopwords and for word lemmatization.

In [1]:
import textract
import numpy as np
import scipy
import gensim
import os
import pandas as pd
import re
import math
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
#nltk.download('averaged_perceptron_tagger')
from collections import Counter
from matplotlib import pyplot as plt
from gensim import corpora, models
%matplotlib inline

## Definition of FocusGroup class 
### Instantiation
- By giving the name of the word file. The word file should have the same format as the focus group documents, that is, each paragraph should be preceeded by a line specifying the name (e.g. Parent 1) of the currently speaking person.
### Attributes
- raw_text: The raw text from the Word document.
- parent_moderator_discussion: The part of raw_text which refers to the discussion between parents and moderators. The rationale for separating the parent_moderator_discussion and within_moderator_discussion attributes is that there was a case when there was a discussion only between the moderators after the discusion between parents and moderators.
- text_including_parents: An np.array storing the discussion between the parents and moderators. Each element of the np.array contains a paragraph from the discussion.
- talkers_including_parents: An np. array with an identical size as text_including_parents containing the respective talker's name (e.g. Parent 1).
- within_moderator_discussion: The part of raw_text which refers to the discussion only moderators, if available. This part of the text was separated from the parent / moderator discussion part of the text by two blank lines.
- text_only_moderators: An np.array storing the discussion only between the moderators, if available. Each element of the np.array contains a paragraph from the discussion.
- talkers_only_moderators: An np. array with an identical size as text_only_moderators containing the respective talker's name (e.g. Moderator 1).
- parent_list: List of unique parent participants.
- moderator_list: List of unique moderator participants.
### Methods
- get_participant_text(participant): gets the list of paragraphs which belong to the given participant.

In [6]:
class FocusGroup:
    def __init__(self, filename):
        self.raw_text=str(textract.process('Data/FocusGroups/' + filename + ".docx")).replace('b\'', '').replace('\'', '')
        
        self.parent_moderator_discussion=self.raw_text.split('\\n\\n\\n')[0].split('\\n\\n')
        self.text_including_parents=np.array([parent_moderator_actual
                                    for parent_moderator_actual in self.parent_moderator_discussion 
                                    if not (('Parent'==re.sub(r" [0-9]:","",parent_moderator_actual)) or 
                                        ('Moderator'==re.sub(r" [0-9]:","",parent_moderator_actual)) or 
                                        ('Administrator'==re.sub(r" [0-9]:","",parent_moderator_actual)) or
                                        ('Speaker'==re.sub(r" [0-9]:","",parent_moderator_actual)))])
        self.talkers_including_parents=np.array([parent_moderator_actual.replace(':', '') 
                                    for parent_moderator_actual in self.parent_moderator_discussion 
                                    if (('Parent'==re.sub(r" [0-9]:","",parent_moderator_actual)) or 
                                        ('Moderator'==re.sub(r" [0-9]:","",parent_moderator_actual)) or 
                                        ('Administrator'==re.sub(r" [0-9]:","",parent_moderator_actual)) or
                                        ('Speaker'==re.sub(r" [0-9]:","",parent_moderator_actual)))])
        
        if len(self.raw_text.split('\\n\\n\\n'))>1:
            self.within_moderator_discussion=self.raw_text.split('\\n\\n\\n')[1].split('\\n\\n')
            self.text_only_moderators=np.array([parent_moderator_actual
                                    for parent_moderator_actual in self.within_moderator_discussion 
                                    if not (('Parent'==re.sub(r" [0-9]:","",parent_moderator_actual)) or 
                                        ('Moderator'==re.sub(r" [0-9]:","",parent_moderator_actual)) or 
                                        ('Administrator'==re.sub(r" [0-9]:","",parent_moderator_actual)) or
                                        ('Speaker'==re.sub(r" [0-9]:","",parent_moderator_actual)))])
            self.talkers_only_moderators=np.array([parent_moderator_actual.replace(':', '') 
                                    for parent_moderator_actual in self.within_moderator_discussion 
                                    if (('Parent'==re.sub(r" [0-9]:","",parent_moderator_actual)) or 
                                        ('Moderator'==re.sub(r" [0-9]:","",parent_moderator_actual)) or 
                                        ('Administrator'==re.sub(r" [0-9]:","",parent_moderator_actual)) or
                                        ('Speaker'==re.sub(r" [0-9]:","",parent_moderator_actual)))])
        
        self.parent_list=[participant for participant in set(self.talkers_including_parents) if 'Parent' in participant]
        self.moderator_list=[participant for participant in set(self.talkers_including_parents) if 'Moderator' in participant]
        
        
    def get_participant_text(self, participant):
        if 'Parent' in participant:
            mask=[member==participant for member in self.talkers_including_parents]
            return list(self.text_including_parents[mask])
        elif 'Moderator' in participant:
            mask=[member==participant for member in self.talkers_including_parents]
            text_from_parent_discussion=self.text_including_parents[mask]
            
            if len(self.raw_text.split('\\n\\n\\n'))==1:
                return list(text_from_parent_discussion)
            else:
                mask=[member==participant for member in self.talkers_only_moderators]
                text_from_moderator_discussion=self.text_only_moderators[mask]
                return list(text_from_parent_discussion) + list(text_from_moderator_discussion)

## Functions to process text
- The original list of stopwords was augmented by stopwords which are filler words (for example, okay) or are the consequences of the automated transcription (for example, inaudible), this extra list of stopwords was saved under the custom_stopwords list.
- The WordNetLemmatizer() class of the nltk library was used for lemmatization.
- The following data processing steps are performed by the text_processing_pipeline function: 
    - Making the string lowercase
    - Removal of punctuation
    - Tokenization
    - Removal of text with less than min_token_count tokens
    - Removing stopwords
    - Lemmatization
    - Removing stopwords (also after the lemmatization)
- The output of the text processing pipeline is a list with the elements, the first element is the processed, tokenized text and the second element is the original text with the purpose to help with the intepretation of the results.

In [7]:
stopwords_list=stopwords.words('english')
custom_stopwords=['go','parent','say','0','yeah','would','okay','start','also','well','u','thank','inaudible','crosstalk','able','hear','actually','hi','oh','definitely','part','anything','sure','anyone','yes','thanks','everything','end','everybody','tand','administrator','whatever','sound','ti','moderator','though','mute','speak','silence','finish','bye','audio']
stopwords_list=stopwords_list+custom_stopwords
remove_stopwords_function=lambda tokenized_text, stopwords: [word for word in tokenized_text if word not in stopwords]
lemmatizer_instance=WordNetLemmatizer()
pos_tags_lemmatize_mapping_dict={'N': 'n', 'V': 'v', 'J': 'a', 'R': 'r'}

def pos_mapping_function(pos_tag, dictionary=pos_tags_lemmatize_mapping_dict):
    if pos_tag[0] in ['N', 'V', 'J', 'R']:
        return dictionary[pos_tag[0]]
    else:
        return 'n'
    
def lemmatizer_function(text, dictionary=pos_tags_lemmatize_mapping_dict, pos_mapping_function=pos_mapping_function,
                       lemmatizer=lemmatizer_instance):
    pos_tags_for_lemmatize=[(word, pos_mapping_function(pos_tag)) for word, pos_tag in nltk.pos_tag(text)]
    pos_tags_lemmatized=[lemmatizer_instance.lemmatize(word, pos=pos_tag) for word, pos_tag in pos_tags_for_lemmatize]
    return pos_tags_lemmatized

def text_processing_pipeline(text_list,additional_stopwords, min_token_count=1, stopwords_list=stopwords_list, 
                             lemmatizer_function=lemmatizer_function, dictionary=pos_tags_lemmatize_mapping_dict,
                             pos_mapping_function=pos_mapping_function, lemmatizer=lemmatizer_instance):
    stopwords_list=stopwords_list+additional_stopwords
    lowercase_text_list=[text.lower() for text in text_list] #Making text lowercase
    lowercase_text_list=[re.sub(r"[^a-zA-Z0-9]", " ", text) for text in lowercase_text_list] #Removal of punctuation
    lowercase_text_list=[text.split() for text in lowercase_text_list] #Tokenization
    filtering_original_text=[text_list[i] for i in range (len(lowercase_text_list)) if len(lowercase_text_list[i])>min_token_count]
    lowercase_text_list=[text for text in lowercase_text_list if len(text)>min_token_count] #Keeping text with an at least a pre-defined token count
    lowercase_text_list=[remove_stopwords_function(text, stopwords_list) for text in lowercase_text_list] #Removing stopwords
    lowercase_text_list=[lemmatizer_function(text) for text in lowercase_text_list] #Lemmatization
    lowercase_text_list=[remove_stopwords_function(text, stopwords_list) for text in lowercase_text_list] #Removing stopwords
    return lowercase_text_list, filtering_original_text

## Process the word data
- Loop over the forteen Word documents with the text processing function and save the result in a list with 15 elements.
- The below code cell contains four lists of additional stopwords for Gaming group / Low PIU group / Media group and Social group, respectively, this set of additional stopwords was generated by Module 2 by iteratively running the gensim LDA algorithm and excluding the words which were included in at least 3 of the 5 topics. The purpose of this data processing step is to avoid having the same set of words in all topics.
- The min_token_count of the text_processing_pipeline function was set to 60, so only paragraphs with at least 60 tokens were kept in the dataset.

In [8]:
file_list=['Gaming_Group1', 'Gaming_Group2', 'Gaming_Group3', 'Gaming_Group4',
           'LowPIU_Group1', 'LowPIU_Group2', 'LowPIU_Group3',
           'Media_Group1', 'Media_Group2', 'Media_Group3', 'Media_Group4',
           'Social_Group1', 'Social_Group2', 'Social_Group3', 'Social_Group4']
additional_stopword_counts=list(dict(Counter([re.sub('[0-9]', '', file,) for file in file_list])).values())
Gaming_group_stopwords=['like', 'get', 'school', 'hour', 'day', 'even', 'think', 'thing', 'way', 'know', 'year', 'week', 'really', 'one',
                       'kid', 'game', 'use', 'time', 'want', 'play', 'much', 'back']
Low_PIU_group_stopwords=['school', 'like', 'time', 'get', 'think', 'kid', 'really',
                        'thing', '00', 'technology', 'year', 'child', 'back', 'lot',
                        'even', 'know', 'want', 'old', 'one']
Media_group_stopwords=['like', 'thing', 'get', 'really', 'kid', 'time', 'want',
                      'school', 'think', 'know', 'one', 'use',
                      'year', 'much', 'back', 'work', 'person', 'pandemic',
                      'see', 'lot', 'good', 'little', 'day', 'old']
Social_group_stopwords=['like', 'get', 'think', 'know', 'thing', 'time', 'school',
                       'really', 'child', 'see', 'want',
                       'kid', 'one', 'lot', 'even']
additional_stopwords_list=[Gaming_group_stopwords, Low_PIU_group_stopwords, Media_group_stopwords, Social_group_stopwords]
additional_stopwords_list=[[stopword_list]*count for count, stopword_list in zip(additional_stopword_counts, additional_stopwords_list)]
additional_stopwords_list=[stopword for additional_stopword in additional_stopwords_list for stopword in additional_stopword]
all_focusgroup_text=[FocusGroup(focus_group_file) for focus_group_file in file_list]
all_focusgroup_processed_text=[text_processing_pipeline(focus_group.text_including_parents,additional_stopword_list, min_token_count=60) for focus_group, additional_stopword_list in zip(all_focusgroup_text, additional_stopwords_list)]

In [9]:
len(all_focusgroup_processed_text)

15